In [1]:
import numpy as np
import pandas as pd
import nltk.corpus
from nltk.corpus import stopwords
from cleantext import clean
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import nltk
import re
import math
from collections import Counter
import json


# 2. Search Engine

### Pre-processing

In [2]:
stop = stopwords.words('english') #stopwords in english 
ps = PorterStemmer() #takes the stem of words
def clean_text(text): 
    text.lower  #case normalization
    text = re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", text)  #removing unicode characters
    text = " ".join([word for word in text.split() if word not in stop and word.isalpha() == True]) #removing stopwords and 'numerical term
    token = word_tokenize(text) #tokenization
    text = [ps.stem(t) for t in token] #stemming and removing non english words
    return(" ".join(text))

## 2.1. Conjunctive query
### 2.1.1 Create your index!


In [3]:
df_for_last=pd.read_csv(r'C:\Users\PC\Desktop\places.tsv', sep='\t') # reading the places with pandas
df=df_for_last.copy()
df=df[0:20]#temporary

for x,i in enumerate(df.loc[:,"placeDesc"]): #cleaning all text in  column
    df.loc[:,"placeDesc"][x]=clean_text(i)
    
word_list=[] 
string=""
for i in df.loc[:,"placeDesc"]: #string containing all words in the description descriptions to 
    string=string+i+" "
word_list.append(string) 
word_list=word_list[0].split()# getting wordlist out of the list and splitting all words and putting into a set to remove duplicates
word_list_set=set(word_list)



C:\Users\PC\AppData\Local\Temp\ipykernel_6480\3779950365.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,"placeDesc"][x]=clean_text(i)


In [4]:
#a=Counter(word_list)
#word frequency


### Creating a dictionary 

In [5]:
dictionary={i[1]:f"termId_{i[0]}" for i in enumerate(word_list_set)}#creates a dictionary from the set of words with term ids
dict(list(dictionary.items())[0:10])

{'noir': 'termId_0',
 'impos': 'termId_1',
 'granit': 'termId_2',
 'photo': 'termId_3',
 'treat': 'termId_4',
 'poster': 'termId_5',
 'stretch': 'termId_6',
 'landmark': 'termId_7',
 'old': 'termId_8',
 'beard': 'termId_9'}

### Creating an inverted index

In [6]:
inverted_index={i:[] for i in dictionary.values()}
for x,y in enumerate(df.loc[:,"placeDesc"]):
    for i in y.split():
        if f"document_{x}" not in inverted_index[dictionary[i]]:
            inverted_index[dictionary[i]].append(f"document_{x}")
dict(list(inverted_index.items())[0:5])

{'termId_0': ['document_4'],
 'termId_1': ['document_1'],
 'termId_2': ['document_3'],
 'termId_3': ['document_10', 'document_12', 'document_13'],
 'termId_4': ['document_18']}

#### Saving inverted index into a json file

In [7]:
with open(r"C:\Users\PC\Desktop\data.json", 'w') as fp: 
    json.dump(dictionary, fp)

### 2.1.2 Execute the query

In [8]:
query_input='new york'
#query_input=input(print("query"))

documents which include all words in the query input

In [9]:
return_list=[]
for i in query_input.split():
    term=dictionary[i]
    return_list.append(set(inverted_index[term]))
inter=[i for i in set.intersection(*return_list)]
inter

['document_18', 'document_0', 'document_19']

### results of documents with query 

In [10]:
df_for_last.loc[[int(i.split("_")[1]) for i in inter],["placeName","placeDesc","placeURL"]]

,placeName,placeDesc,placeURL
18,Roosevelt Island Smallpox Hospital Ruins,Few diseases have had a greater impact on the ...,https://www.atlasobscura.com/places/roosevelt-...
0,City Hall Station,The first New York City subway was built and o...,https://www.atlasobscura.com/places/city-hall-...
19,Eastern State Penitentiary,Thrust into the middle of urban Philadephia is...,https://www.atlasobscura.com/places/eastern-st...


# 2.2) Conjunctive query & Ranking score


## 2.2.1) Inverted index


In [11]:
word_list[:10]

['the',
 'first',
 'new',
 'york',
 'citi',
 'subway',
 'built',
 'oper',
 'interborough',
 'rapid']

#### TF's of documents

In [12]:
TFS=[]
for i in df.loc[:,"placeDesc"]:
    tfs={}
    dene2=i.split()
    for x in dene2:
        tfs[x]=dene2.count(x)/len(dene2)
    TFS.append(tfs)
TFS
document_TFs=pd.DataFrame(TFS)
document_TFs = document_TFs.fillna(0)

document_TFs[:10]


,the,first,new,york,citi,subway,built,oper,interborough,rapid,...,steve,buscemi,penitentiari,upris,winter,bastil,celebr,halloween,tourobscura,april
0,0.010204,0.005102,0.015306,0.015306,0.030612,0.005102,0.005102,0.005102,0.005102,0.005102,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.003390,0.003390,0.000000,0.000000,0.006780,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.017241,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.017544,0.000000,0.000000,0.000000,0.005848,0.000000,0.011696,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.010526,0.002632,0.000000,0.000000,0.002632,0.000000,0.002632,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.011236,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.000000,0.000000,0.000000,0.000000,0.017751,0.005917,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.020833,0.000000,0.000000,0.000000,0.010417,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.016949,0.000000,0.033898,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
new_inverted_index={i:[] for i in dictionary.values()}#empty inverted indexes
for x,y in enumerate(df.loc[:,"placeDesc"]):
    splitted=y.split()
    splitted_tekrarlamaz=set(splitted)
    for i in splitted_tekrarlamaz:         
        new_inverted_index[dictionary[i]].append([x,f"tfidf={document_TFs.loc[x][i]}"])
new_inverted_index
dict(list(new_inverted_index.items())[0:5])

{'termId_0': [[4, 'tfidf=0.002631578947368421']],
 'termId_1': [[1, 'tfidf=0.003389830508474576']],
 'termId_2': [[3, 'tfidf=0.005847953216374269']],
 'termId_3': [[10, 'tfidf=0.01680672268907563'],
  [12, 'tfidf=0.00684931506849315'],
  [13, 'tfidf=0.005714285714285714']],
 'termId_4': [[18, 'tfidf=0.009009009009009009']]}

## 2.2.2) Execute the query


#### Creating relevant dataframe for calculating similarity of query to the document

In [14]:
# for every word in query its term_ID is learned from the dictionary and corresponding document numbers in inverted index are
#added to the "rdocs".These are the documents which has the query words in them.

query_input="city subway"
query_input=clean_text(query_input)
return_list=[]
for i in query_input.split():
    term=dictionary[i]
    return_list.append(set(inverted_index[term]))
return_list=[i for i in set.intersection(*return_list)]

rdocs=return_list
rdocs_numbers=[int(i.split("_")[1]) for i in rdocs]

#### Number of documents that has the query words

In [15]:
dictionary_of_words_query={}
for word in query_input.split():
    count=0
    for document in df.loc[:,"placeDesc"]:
        if word in document.split():
            count+=1
    dictionary_of_words_query[word]=count
dictionary_of_words_query

    
    

{'citi': 9, 'subway': 2}

#### TF-IDF's of relevant words in documents

In [26]:
rTFIDFS={}
for i in query_input.split():
    rTFIDFS[i]=[]
    tfidfs=[]
    dene2=i.split()
    for x in rdocs_numbers:
        rTFIDFS[i].append(df.loc[:,"placeDesc"][x].count(i)/len(df.loc[:,"placeDesc"][x].split())*(1+math.log(len(df.loc[:,"placeDesc"])/dictionary_of_words_query[i])))
document_TFIDFs=pd.DataFrame(rTFIDFS, index=rdocs_numbers)
document_TFIDFs[:5]


,citi,subway
7,0.031926,0.019542
0,0.055056,0.016850


In [17]:
#new dataframe for documents including query

relevant_df=document_TFIDFs.loc[rdocs_numbers][query_input.split()]


In [24]:
relevant_df[:10]

,citi,subway
7,0.031926,0.019542
0,0.055056,0.016850


#### Calculating the similarity between query and documents

In [19]:
dictionary_of_words_query={}
for word in relevant_df:
    count=0
    for document in df.loc[:,"placeDesc"]:
        if word in document.split():
            count+=1
    dictionary_of_words_query[word]=count
dictionary_of_words_query

    
    

{'citi': 9, 'subway': 2}

In [20]:
#TFIDF of query
query_tfIDFs=[]
for x in query_input.split():
    query_tfIDFs.append(query_input.split().count(x)/len(query_input.split())*  (1+math.log(len(query_input.split())/dictionary_of_words_query[x])))
query_tfIDFs


[-0.2520386983881371, 0.5]

In [21]:
#||Query||
add=0
for i in query_tfIDFs:
    add+=i*i    
Query_L=math.sqrt(add)   

#Similarities of all documents
similarity_list=[]
for i in relevant_df.index:
    add1=0
    for x in relevant_df.loc[i]:
        add1+=x*x    
    Document_L=math.sqrt(add1)   
    similarity_list.append((np.dot(relevant_df.loc[i],query_tfIDFs))/Query_L*Document_L)
    #similarity_list.append([i,(np.dot(relevant_df.loc[i],query_tfIDFs))/Query_L*Document_L])

sorted(similarity_list)

[-0.0005605578834324423, 0.00011527378617234834]

#### results

In [22]:
sonuc_new=df_for_last.loc[rdocs_numbers,["placeName","placeDesc","placeURL"]]
sonuc_new.insert(3,"similarity",sorted(similarity_list))

#add similarity list at the end of the dataframe

In [23]:
sonuc_new=sonuc_new.sort_values("similarity",ascending=False)
sonuc_new

,placeName,placeDesc,placeURL,similarity
0,City Hall Station,The first New York City subway was built and o...,https://www.atlasobscura.com/places/city-hall-...,0.000115
7,Underground Tunnels of Los Angeles,While the rest of the country was forced to go...,https://www.atlasobscura.com/places/undergroun...,-0.000561
